In [5]:
import Table_Reader as TR
import LCCall as LCC
import LCAnalyses as LCA
import LCPlot as LCP
import importlib
import matplotlib.pyplot as plt
import scipy.special as fc
import random as rd
import numpy as np
importlib.reload(LCC)
importlib.reload(LCA)
importlib.reload(LCP)
importlib.reload(TR)

<module 'Table_Reader' from '/media/lyhee/Secdrive/TSmain/Pip_test/trigger_codes_github/Table_Reader.py'>

In [2]:
print(fc.gammainc(0.5,0.5*1**2))
print(fc.gammaincinv(0.5,0.8))

0.6826894921370859
0.821187207574909


In [4]:
fmean = 1.4
np.sqrt(0.014**2 + (fmean*0.02)**2)/fmean

0.0223606797749979

In [6]:
input_dir = "/mnt/Secdrive/TSmain/TSmain/"
rgs = ['IC348','NGC1333','NGC2024','NGC2068','OMC23','OPHCORE','SERPM','SERPS']
rra  =[56.075, 52.225, 85.42083, 86.55417, 83.87917, 246.77083, 277.45417, 277.50833]
rdec =[32.08306, 31.28111, -1.8975, -0.10139, -5.01056, -24.54361, 1.25556, -2.04667]
#indices = np.where(np.sqrt(np.array(data.RAs)**2+np.array(data.DECs)**2)/60 < 18)
%matplotlib tk

In [8]:
freq_in = np.array([0.1, 10])
A_fid_in = np.array([np.log10(0.5),np.log10(5)])
phase_in = np.array([0,2*np.pi])


# 5-10 sec for SERPM for 1000 iter
# ~0.6 min for OPHCORE for 10000 iter

In [9]:
writing=1
test = 0
rgn = 6
query = []
Rst=[]

#L = [k,freq,phase/2/np.pi,H_mean[-1],T1,T2,T3*2,A_n[0],A_n[1],A_n[2],chi_Null]
for rgn in range(1,2):
    #for freq in [0.1, 0.15, 0.2, 0.5,1,5]:
    for period in [10,]:
        if writing:
            file = open(input_dir+"Completeness/"+rgs[rgn]+"_period"+str(period)+"_new.tab",'w')
            file.write("#Index"+'\t'+ "Freq"+'\t'+"Phase"+'\t'+'P'+'\t'+"C"+'\t'+
                   "T1"+'\t'+"T2"+'\t'+"T3"+'\t'+"A2"+'\t'+"A3"+'\t'+'\t'+"A4"+'\t'+
                   "chi2_null"+'\t'+ "chi2_null_det"+"\n")
        Index = 1
        counter = 1
        plot = 0
        Iter = 10000
        thr = np.array([1e-2,1e-3,1e-4])
        output_dir = input_dir+"IfyouNeed/"
        for i in range(rgn,rgn+1):
            print(rgs[i])
            data = TR.ReadTable(input_dir+'Data/'+rgs[i]+'_source_data.dat')
            indices = [int(x) for x in data.Indices]
            data.JDs = (np.array(data.JDs) - data.JDs[0])/365.25
            for k in range(0,Iter):
                #freq = rd.uniform(freq_in[0],freq_in[1])
                #freq = 0.15
                
                #r_period = (np.max(data.JDs)-np.min(data.JDs))/freq
                #freq = (np.max(data.JDs)-np.min(data.JDs))/period
                freq = 1/period
                r_period=period
                #A_fid = np.exp(rd.uniform(A_fid_in[0],A_fid_in[1]))
                phase = rd.uniform(phase_in[0],phase_in[1])

                j = 0

                #phases = np.linspace(0,2*np.pi,10)[:-1]

                #modelpar = [mean[jy/beam], freq[day], amp[jy/beam], iphase]

                #fid_err = np.sqrt(0.014**2+(0.02*data.mean_pfluxes[j])**2)

                num = len(data.pfluxes[j])
                H_data = []
                G = []
                H_real = []
                H_mean = []
                for x in data.JDs:
                    G.append(rd.gauss(0,1))
                    H_real.append(np.sin(2*np.pi*freq*x+phase))
                    H_data.append(H_real[-1]+G[-1])
                data.pfluxes[j] = np.array(H_data)
                H_real = np.array(H_real)
                H_mean.append(np.mean(H_real))
                H_real = H_real-np.mean(H_real)-np.mean(G)
                G = np.array(G)
                T3 = np.sum(G**2)/num
                T1 = np.sum((H_real)**2)/num
                T2 = np.sum(G*H_real)/num
                #T4 = A_fid


                if not test:
                    #Null Hypothesis
                    chi2_Null = np.sum(abs(data.pfluxes[j]-np.mean(data.pfluxes[j]))**2)

                    #Periodogram
                    ls_source = LCA.LCLombscargle(data.JDs,data.pfluxes[j],1)
                    modelpar = ls_source.modelpara()
                    FAP_Sin = ls_source.FAPs[np.where(ls_source.powers == np.max(ls_source.powers))][0]
                    JD_center = (min(data.JDs) + max(data.JDs)) / 2
                    Power_Sin = np.max(ls_source.powers)
                    #P_Null = fc.gammainc((num-1)/2, chi2_Null/2)

                    FAP_Sin_single = ls_source.FAP_single
                    #modelpar = [mean[jy/beam], freq[day], amp[jy/beam], iphase]
                    #sinfit = ls_source.mulsin(data.JDs, modelpar)
                    #P_Sin = fc.gammainc((num-3)/2, chi2_Sin/2)
                    #data_Sin = np.array(data.pfluxes[j])-sinfit
                    #period = np.round(1/modelpar[1])
                    freq_min = np.min(ls_source.freqs)
                    freq_max = np.max(ls_source.freqs)
                    dfreq = ls_source.freqs[1]-ls_source.freqs[0]

                    num_freqs=int(freq/dfreq)+1
                    alphas= (thr/num_freqs)**(-2/(num-3))
                    A_n = []
                    for alpha in alphas:
                        a = T1
                        b = T2*2
                        c = T3*(1-alpha)
                        A_n.append((-b+np.sqrt(b**2-4*a*c))/(2*a))

                    #Null Hypothesis when detected
                    chi2_Null_det = a*A_n[-1]**2+b*A_n[-1]+c
                    #if any(sum([checker1,checker2,checker3])): writing =1


                L = [k, freq, phase/2/np.pi 
                     ,r_period, H_mean[-1], T1
                     , T2*2, T3, A_n[0]
                     , A_n[1], A_n[2], chi2_Null/num,chi2_Null_det/num]
                Rst.append(L)
                L = [str(x)+'\t' for x in L[:-1]]+[str(L[-1])]
                if writing:
                    file.writelines(L)
                    file.write('\n')
                Index += 1
            if writing:
                file.close()
            print(freq)
            print("DONE!")
#query = set(query)

NGC1333
2016-02-05T
2017-12-08T
2018-07-07T
0.1
DONE!


In [10]:
#(1e-3/num_freqs)**(-2/(num-3))
x= np.linspace(10,50,41)
plt.close()
plt.plot(x,[(0.001/20)**(-2/(t-3)) for t in x])

In [50]:
1e-3

0.001

In [298]:
H_data = []
G = []
H_real = []
H_mean = []
for x in data.JDs:
    G.append(rd.gauss(0,1))
    H_real.append(5*np.sin(2*np.pi*(0.1*x+0.0)))
    H_data.append(H_real[-1]+G[-1])
data.pfluxes[j] = np.array(H_data)

plt.figure()
plt.scatter(data.JDs,data.pfluxes[j])
plt.scatter(data.JDs,H_real)

In [11]:
output_dir = input_dir+'Completeness/'
rgn=5
a1 = np.loadtxt(output_dir + rgs[rgn] +"_freq0.1.tab")
a2 = np.loadtxt(output_dir + rgs[rgn] +"_freq0.15.tab")
a3 = np.loadtxt(output_dir + rgs[rgn] +"_freq0.2.tab")
a4 = np.loadtxt(output_dir + rgs[rgn] +"_freq0.5.tab")
a5 = np.loadtxt(output_dir +  rgs[rgn] +"_freq1.tab")
a6 =np.loadtxt(output_dir + rgs[rgn] +"_freq5.tab")
a1 = TR.Columnread(a1)
a2 = TR.Columnread(a2)
a3 = TR.Columnread(a3)
a4 = TR.Columnread(a4)
a5 = TR.Columnread(a5)
a6 = TR.Columnread(a6)

In [13]:
output_dir = input_dir+'Completeness/'
rgn=6
a1 = np.loadtxt(output_dir + rgs[rgn] +"_period12.tab")
a2 = np.loadtxt(output_dir + rgs[rgn] +"_period8.tab")
a3 = np.loadtxt(output_dir + rgs[rgn] +"_period6.tab")
a4 = np.loadtxt(output_dir + rgs[rgn] +"_period4.tab")
a5 = np.loadtxt(output_dir +  rgs[rgn] +"_period2.tab")
a6 =np.loadtxt(output_dir + rgs[rgn] +"_period1.tab")
a7 =np.loadtxt(output_dir + rgs[rgn] +"_period40.tab")
a8 =np.loadtxt(output_dir + rgs[rgn] +"_period24.tab")
a1 = TR.Columnread(a1)
a2 = TR.Columnread(a2)
a3 = TR.Columnread(a3)
a4 = TR.Columnread(a4)
a5 = TR.Columnread(a5)
a6 = TR.Columnread(a6)
a7 = TR.Columnread(a7)
a8 = TR.Columnread(a8)
a1_thr = np.sort(a1[8])[9900]
a2_thr = np.sort(a2[8])[9900]
a3_thr = np.sort(a3[8])[9900]
a4_thr = np.sort(a4[8])[9900]
a5_thr = np.sort(a5[8])[9900]
a6_thr = np.sort(a6[8])[9900]
a7_thr = np.sort(a7[8])[9900]
a8_thr = np.sort(a8[8])[9900]

In [144]:
a5[1][0]

0.2

In [215]:
plt.close()

a = TR.Columnread(Rst)
fig, ax = plt.subplots()
ax.hist2d(a1[2],a1[8])

(array([[  6., 106., 328., 298., 138.,  70.,  28.,   9.,   5.,   0.],
        [  9., 162., 363., 276., 119.,  56.,  18.,  10.,   2.,   0.],
        [206., 617., 178.,  21.,   1.,   0.,   0.,   0.,   0.,   0.],
        [390., 537.,  54.,   2.,   0.,   0.,   0.,   0.,   0.,   0.],
        [167., 615., 227.,  35.,   2.,   0.,   0.,   0.,   0.,   0.],
        [  1., 116., 322., 280., 148.,  62.,  21.,   8.,   1.,   2.],
        [  2., 144., 344., 239., 127.,  54.,  15.,   6.,   1.,   2.],
        [175., 600., 184.,  22.,   1.,   0.,   0.,   0.,   0.,   0.],
        [401., 606.,  64.,   1.,   0.,   0.,   0.,   0.,   0.,   0.],
        [158., 600., 203.,  29.,   6.,   0.,   0.,   0.,   0.,   0.]]),
 array([1.70610650e-04, 1.00137706e-01, 2.00104802e-01, 3.00071897e-01,
        4.00038993e-01, 5.00006088e-01, 5.99973184e-01, 6.99940279e-01,
        7.99907375e-01, 8.99874471e-01, 9.99841566e-01]),
 array([0.41719084, 1.21032028, 2.00344972, 2.79657916, 3.5897086 ,
        4.38283804, 5.175967

In [14]:
plt.close()
fig, ax = plt.subplots()
ax.set_xlabel('Phase')
ax.set_ylabel('Sigma Level')
ax.scatter(a1[2],a1[8],alpha=0.3,edgecolors='none',label=r'P=12')
ax.scatter(a3[2],a3[8],alpha=0.3,edgecolors='none',label=r'P=8')
ax.scatter(a5[2],a5[8],alpha=0.3,edgecolors='none',label=r'P=2')
ax.legend()

In [152]:
a6_thr = np.sort(a6[8])[9900]
print(a6_thr)

2.14381044354215


In [15]:
fig, ax = plt.subplots(2,2)

x = a7[2]
y = a7[8]
x2 = a8[2]
y2 = a8[8]
x3 = a1[2]
y3 = a1[8]
x4 = a4[2]
y4 = a4[8]

ax[0][0].scatter(x, y,marker='o',color='grey',edgecolors='none',alpha=0.2,s=20
           ,label=r'P = 40 yr')
ax[0][1].scatter(x2, y2,marker='o',color='orange',edgecolors='none', alpha=0.2,s=20
          ,label=r'P = 24 yr')
ax[1][0].scatter(x3, y3,marker='o',color='green',edgecolors='none', alpha=0.2,s=20
          ,label=r'P = 12 yr')
ax[1][1].scatter(x4, y4,marker='o',color='lightsteelblue',edgecolors='none', alpha=0.2,s=20
          ,label=r'P = 4 yr')

for axis in ax:
    axis[0].label_outer()
    axis[1].label_outer()
    #axis[0].annotate(r"Sub-mm",xy=[0.35,0.8],xycoords='figure fraction',fontsize=15
    #axis[0].set_ylabel(r"Detectable Amp. [$\sigma$]")
    #axis[1].set_ylabel(r"Detectable Amp. [$\sigma$]")
    axis[0].legend()
    axis[1].legend()
plt.tight_layout()

In [256]:
axis

array([<matplotlib.axes._subplots.AxesSubplot object at 0x7f656e772090>,
      dtype=object)

In [16]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
# the random data

x = a1[2]
y = a1[8]
x2 = a2[2]
y2 = a2[8]
x3 = a3[2]
y3 = a3[8]
x4 = a5[2]
y4 = a5[8]


x = a7[2]
y = a7[8]
y = a7[6]

x2 = a8[2]
y2 = a8[8]
x3 = a1[2]
y3 = a1[8]
x4 = a4[2]
y4 = a4[8]

thr1 = np.sort(y)[9900]
thr1_2 = np.sort(y)[6800]
thr2 = np.sort(y2)[9900]
thr3 = np.sort(y3)[9900]
thr4 = np.sort(y4)[9900]
print(a1_thr,a2_thr,a3_thr)
fig, ax = plt.subplots(figsize=(8, 8))

# the scatter plot:
ax.scatter(x, y,marker='o',color='grey',edgecolors='none',alpha=0.2,s=20
           ,label=r'P = 40 yr')
ax.scatter(x2, y2,marker='o',color='orange',edgecolors='none', alpha=0.2,s=20
          ,label=r'P = 24 yr')
ax.scatter(x3, y3,marker='o',color='green',edgecolors='none', alpha=0.2,s=20
          ,label=r'P = 12 yr')
ax.scatter(x4, y4,marker='o',color='lightsteelblue',edgecolors='none', alpha=0.2,s=20
          ,label=r'P = 4 yr')
ax.plot([-0.01,1.01],[thr1,thr1],'2--',color='grey')
ax.plot([-0.01,1.01],[thr2,thr2],'2--',color='orange')
ax.plot([-0.01,1.01],[thr3,thr3],'2--',color='green')
ax.plot([-0.01,1.01],[thr4,thr4],'2--',color='blue')
#ax.scatter()

ax.set_xlim(-0.01,1.02)
ax.set_ylim(0,20)
ax.set_xlabel(r"Phase",fontsize=14)
ax.set_ylabel(r"Detectable Amplitude ($\sigma$)",fontsize=14)
ax.legend(loc='upper right')
ax.tick_params(labelsize=13)

# Set aspect of the main axes.
#ax.set_aspect(0.2)

# create new axes on the right and on the top of the current axes
divider = make_axes_locatable(ax)
# below height and pad are in inches
#ax_histx = divider.append_axes("top", 1.0, pad=0.1, sharex=ax)
ax_histy = divider.append_axes("right", 1.0, pad=0.2, sharey=ax)

# make some labels invisible
#ax_histx.xaxis.set_tick_params(labelbottom=False)
#ax_histx.xaxis.set_xscale("log")
ax_histy.yaxis.set_tick_params(labelleft=False)
#ax_histy.yaxis.set_yscale("log")

# now determine nice limits by hand:
#binwidth = 0.05
binwidth = 0.3
xymax = max(np.max(np.abs(x)), np.max(np.abs(y)))
lim = (int(xymax/binwidth) + 1)*binwidth

bins = np.arange(-lim, lim + binwidth, binwidth)
#ax_histx.hist(x, bins=bins,histtype='step',color='black')
ax_histy.hist(y, bins=bins,orientation='horizontal',histtype='step',color='black'
             ,linewidth=2,density=0)
ax_histy.plot([0,10000],[thr1,thr1],color='black',ls='--')
ax_histy.hist(y2, bins=bins,orientation='horizontal',histtype='step',color='orange'
             ,linewidth=2,density=0)
ax_histy.plot([0,10000],[thr2,thr2],color='orange',ls='--')
ax_histy.hist(y3, bins=bins,orientation='horizontal',histtype='step',color='green'
             ,linewidth=2,density=0)
ax_histy.plot([0,10000],[thr3,thr3],color='green',ls='--')
ax_histy.hist(y4, bins=bins,orientation='horizontal',histtype='step',color='blue'
             ,linewidth=2,density=0)
ax_histy.plot([0,10000],[thr4,thr4],color='blue',ls='--')
#ax_histy.plot([0,5000],[thr2,a4_thr],color='blue')
#ax_histy.plot([0,5000],[thr3,a5_thr],color='violet')

# the xaxis of ax_histx and yaxis of ax_histy are shared with ax,
# thus there is no need to manually adjust the xlim and ylim of these
# axis.
#ax_histx.set_xscale("log")
#ax_histy.set_yscale("log")

#ax_histx.set_yscale("log")
#ax_histx.set_ylim([0.8, 110])
ax_histy.set_xscale("log")
ax_histy.set_xlim([0.8, 10000])
#ax.set_ylim(5e-3,15)
#ax.set_xlim(0.13,15)
#ax_histx.set_ylim(0.13,15)
#ax.plot([0.13,15],[5e-3,15],ls='--',color='grey')
#ax_histy.set_xlim(5e-3,15)
#ax_histx.set_yticks([0, 30, 60])
#ax_histy.set_xticks([0, 30, 60])
plt.show()

4.673801657760634 2.683113712061837 1.9135528455796014


In [22]:
plt.savefig(output_dir+"Phase-DAmp-99per.pdf")

In [17]:
rgn=5
a1 = np.loadtxt(output_dir + rgs[rgn] +"_period12.tab")
a2 = np.loadtxt(output_dir + rgs[rgn] +"_period8.tab")
a3 = np.loadtxt(output_dir + rgs[rgn] +"_period6.tab")
a4 = np.loadtxt(output_dir + rgs[rgn] +"_period4.tab")
a5 = np.loadtxt(output_dir + rgs[rgn] +"_period2.tab")
a6 = np.loadtxt(output_dir + rgs[rgn] +"_period1.tab")
a1 = TR.Columnread(a1)
a2 = TR.Columnread(a2)
a3 = TR.Columnread(a3)
a4 = TR.Columnread(a4)
a5 = TR.Columnread(a5)
a6 = TR.Columnread(a6)
a1_thr = np.sort(a1[8])[9900]
a2_thr = np.sort(a2[8])[9900]
a3_thr = np.sort(a3[8])[9900]
a4_thr = np.sort(a4[8])[9900]
a5_thr = np.sort(a5[8])[9900]
a6_thr = np.sort(a6[8])[9900]
#print(a1_thr,a2_thr,a3_thr,a4_thr,a5_thr)

#a1 freq0.1_test.tab")
#a6 freq0.15_test.tab")
#a5 freq0.2.tab")
#a4 freq0.5.tab")
#a2 freq1.tab")
#a3 freq5.tab")

In [19]:
plt.close()
fig, ax = plt.subplots()
ax.set_xlabel('Period [yr]')
ax.set_ylabel(r'Confident Detection level [$\sigma$]')
ax.set_ylim([1.2,6])
xfreqs =np.array([0.1,0.15,0.2,0.5,1,5])
xperiods = np.array([12,8,6,4,2,1])

for rgn in range(0,8):
    text = ""
    if rgn == 1:
        text = "_test"
    a1 = np.loadtxt(output_dir + rgs[rgn] +"_period12"+text+".tab")
    a2 = np.loadtxt(output_dir + rgs[rgn] +"_period8"+text+".tab")
    a3 = np.loadtxt(output_dir + rgs[rgn] +"_period6"+text+".tab")
    a4 = np.loadtxt(output_dir + rgs[rgn] +"_period4"+text+".tab")
    a5 = np.loadtxt(output_dir + rgs[rgn] +"_period2"+text+".tab")
    a6 = np.loadtxt(output_dir + rgs[rgn] +"_period1"+text+".tab")
    a1 = TR.Columnread(a1)
    a2 = TR.Columnread(a2)
    a3 = TR.Columnread(a3)
    a4 = TR.Columnread(a4)
    a5 = TR.Columnread(a5)
    a6 = TR.Columnread(a6)
    a1_thr = np.sort(a1[8])[9900]
    a2_thr = np.sort(a2[8])[9900]
    a3_thr = np.sort(a3[8])[9900]
    a4_thr = np.sort(a4[8])[9900]
    a5_thr = np.sort(a5[8])[9900]
    a6_thr = np.sort(a6[8])[9900]
    
    a1_thr2 = np.sort(a1[8])[6800]
    a2_thr2 = np.sort(a2[8])[6800]
    a3_thr2 = np.sort(a3[8])[6800]
    a4_thr2 = np.sort(a4[8])[6800]
    a5_thr2 = np.sort(a5[8])[6800]
    a6_thr2 = np.sort(a6[8])[6800]
    
    print(rgs[rgn])
    print([a1_thr,a2_thr,a3_thr,a4_thr,a5_thr,a6_thr])
    print([a1_thr2,a2_thr2,a3_thr2,a4_thr2,a5_thr2,a6_thr2])
    #print(np.where(np.sort(a1[8])<a4_thr)[-1])
    #print(np.where(np.sort(a2[8])<a4_thr)[-1])
    thrs2= np.array([a1_thr2,a2_thr2,a3_thr2,a4_thr2,a5_thr2,a6_thr2])
    thrs = np.array([a1_thr,a2_thr,a3_thr,a4_thr,a5_thr,a6_thr])
    ax.errorbar(xperiods,thrs,fmt='o-',label=rgs[rgn])
ax.legend()

IC348
[4.363723865021905, 2.6564658574561455, 1.963110908145642, 1.8633787432096465, 1.9570742328548625, 2.113938967958012]
[2.0099895532961747, 1.4396597823282016, 1.1920637264717708, 1.1625454074232229, 1.2360382122685971, 1.3267304197998113]


OSError: /mnt/Secdrive/TSmain/TSmain/Completeness/NGC1333_period12_test.tab not found.

In [10]:
plt.savefig(input_dir+"Completeness/"+"map_CDl.pdf")

In [ ]:
for rgn in range(0,8):
    a1 = np.loadtxt(output_dir + rgs[rgn] +"_period12.tab")
    a2 = np.loadtxt(output_dir + rgs[rgn] +"_period8.tab")
    a1 = TR.Columnread(a1)
    a2 = TR.Columnread(a2)

In [139]:
plt.savefig(input_dir+"Completeness/"+"Phase-Amp_diagram_more.pdf")

In [20]:
plt.close()
fig, ax = plt.subplots()
ax.set_xlabel('Chi-null')
ax.set_ylabel('Number')
binwidth = 0.1
xymax = max(np.max(np.abs(x)), np.max(np.abs(y)))
lim = (int(xymax/binwidth) + 1)*binwidth

bins = np.arange(-lim, lim + binwidth, binwidth)
ax.hist(a1[11], histtype='step',color='black'
             ,linewidth=2,label=r'P=10$\tau$')
ax.hist(a2[11], histtype='step',color='orange'
             ,linewidth=2,label=r'P=1$\tau$')
ax.hist(a3[11],histtype='step',color='green'
             ,linewidth=2,label=r'P=0.2$\tau$')
ax.legend()

In [111]:
plt.savefig(input_dir+"Completeness/"+"chi2_null_hist.pdf")

In [21]:
plt.close()
fig, ax = plt.subplots()
x = np.linspace(0.14,1.4,101)

ax.plot(x, np.sqrt(0.014**2+(0.02*x)**2))
ax.plot(x, 2*np.sqrt(0.014**2+(0.02*x)**2))
ax.plot(x, 3*np.sqrt(0.014**2+(0.02*x)**2))

ax.set_xscale('log')

In [110]:
a1[11][-1]

1.1022966728098704